In [3]:
import selenium
import os
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", UserWarning)

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


In [7]:
driver = webdriver.Edge()

In [8]:
driver.get('https://www.youtube.com/watch?v=7NHJtSs0OSc')

In [9]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [15]:
soup.find(itemprop_ ='video-id')

In [ ]:
# driver.find_element_by_class_name('html5-video-container').find_element_by_class_name('video-stream.html5-main-video').click()

'15M'

In [17]:
import urllib.parse as urlparse

def video_id(value):
    """
    Examples:
    - http://youtu.be/SA2iWivDJiE
    - http://www.youtube.com/watch?v=_oPAwA_Udwc&feature=feedu
    - http://www.youtube.com/embed/SA2iWivDJiE
    - http://www.youtube.com/v/SA2iWivDJiE?version=3&amp;hl=en_US
    """
    query = urlparse.urlparse(value)
    if query.hostname == 'youtu.be':
        return query.path[1:]
    if query.hostname in ('www.youtube.com', 'youtube.com'):
        if query.path == '/watch':
            p = urlparse.parse_qs(query.query)
            return p['v'][0]
        if query.path[:7] == '/embed/':
            return query.path.split('/')[2]
        if query.path[:3] == '/v/':
            return query.path.split('/')[2]
    # fail?
    return None

In [19]:
video_id("https://www.youtube.com/watch?v=4sosXZsdy-s&ab_channel=TraversyMedia")

'4sosXZsdy-s'

In [4]:
re.compile("https://www.youtube.com/watch?v=7NHJtSs0OSc")

re.compile(r'https://www.youtube.com/watch?v=7NHJtSs0OSc', re.UNICODE)

In [20]:
del soup

In [45]:
class YouTubeScrapper():
    # Constructor
    def __init__(self,
                    webdriver_path:str='./',
                    browser={'edge', 'chrome'},
                    output_path:str='', headless:bool=True):
        assert browser in {'edge', 'chrome'}, 'browser not comparable'
        self.scrapper_class = 'YouTube scrapper'
        options = webdriver.edge.options.Options() if browser == 'edge' \
                                                else webdriver.chrome.options.Options()
        if headless:
            options.add_argument("--headless")
        try:
            if webdriver_path == './':
                self.driver = webdriver.Edge(options=options) if browser == 'edge' \
                                        else webdriver.Chrome(options=options)
            else:
                self.driver = webdriver.Edge(webdriver_path, 
                                    options=options) if browser=='edge' \
                            else webdriver.Chrome(webdriver_path, options=options) 
        except:
            print("webdriver can't be found with that input path")
        self.output_path = output_path

    def get_video_id(self, link):
        query = urlparse.urlparse(link)
        if query.hostname == 'youtu.be':
            return query.path[1:]
        if query.hostname in ('www.youtube.com', 'youtube.com'):
            if query.path == '/watch':
                p = urlparse.parse_qs(query.query)
                return p['v'][0]
            if query.path[:7] == '/embed/':
                return query.path.split('/')[2]
            if query.path[:3] == '/v/':
                return query.path.split('/')[2]
        # fail?
        return None

    def scroll_to_bottom(self, limit=None, time_sleep:int=1.5):
        last_height = self.driver.execute_script('\
        return window.scrollY')
        current = 1
        limit_check = lambda x: True if x > limit else False
        while True:
            from selenium.webdriver.common.keys import Keys
            html = self.driver.find_element_by_tag_name('html')
            html.send_keys(Keys.END)
            time.sleep(time_sleep)
            new_height = self.driver.execute_script('\
                return window.scrollY')
            if last_height == new_height:
                break
            if limit:
                if limit_check(current):
                    break
                current += 1
            last_height = new_height
        
    def from_string_to_int(self, fol_string: str):
        tens = {'K': 10e2, 'M': 10e5, 'B': 10e8, 'k': 10e2, 'm': 10e5, 'b': 10e8}
        if (fol_string[-1] != 'K' and fol_string[-1] != 'M' 
                        and fol_string[-1] != 'k' and fol_string[-1] != 'm'
                        and fol_string[-1] != 'b' and fol_string[-1] != 'B'):
            return int(fol_string)
        f = lambda x: int(float(x[:-1])*tens[x[-1]])
        return f(fol_string)
    # get likes method
    def get_likes(self) -> int:
        fol_string = self.soup.find(id='menu-container').find(id='text').text
        return self.from_string_to_int(fol_string=fol_string)
    #get views
    def get_views(self) -> int:
        fol_string = self.soup.find_all(
            'div', id='info')[-1].find(
                'div', id='count').find(
                    class_=re.compile(r'short*')).text.split()[0]
        return self.from_string_to_int(fol_string=fol_string)
    #scrape method
    def scrape_comments(self, link: str, save=True,
                            scroll_limit=None, time_sleep:int=3):
        self.driver.get(link)
        time.sleep(time_sleep//2)
        self.soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        del self.driver

        # try:
        #     print('something')
        #     self.driver.find_element_by_class_name(
        #         'html5-video-container').find_element_by_class_name(
        #             'video-stream.html5-main-video').click()
        # except:
        #     print('Video is not playing')
        video_info = {
            'video_id': self.get_video_id(link=link),
            'link': link,
            'likes': self.get_likes(),
            'views': self.get_views(),
        }
        df_video = pd.DataFrame(video_info, index=['video'])
        if save: 
            df_video.to_csv(os.path.join(self.output_path, 'video_info.csv'), index=False)
        # self.scroll_to_bottom(limit=scroll_limit)
        # prev_soup = self.soup
        # self.soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        return df_video

In [46]:
scrapper = YouTubeScrapper(browser='edge')

In [47]:
scrapper.scrape_comments(link='https://www.youtube.com/watch?v=7NHJtSs0OSc')

,video_id,link,likes,views
video,7NHJtSs0OSc,https://www.youtube.com/watch?v=7NHJtSs0OSc,10000,197000
